In [ ]:
## =================================================
## Author: Farid Javadnejad
## Date: 2025-09-25
## Last Update: 2025-09-25
#
## =================================================
#
# DESCRIPTION:
#
# This notebook updates elevation (Z) values in a LandXML surface model using new survey data.
#
# It performs the following steps:
# - Loads original surface points and updated survey data.
# - Matches points using nearest-neighbor search (KDTree).
# - Computes vertical differences (dZ) and filters outliers.
# - Applies cleaned dZ values to compute new Z elevations.
# - Updates the <Pnts> section in the LandXML file with new Z values.
# - Preserves the original surface structure and updates <Surface> name and description.
# - Outputs a new LandXML file for downstream use.
# Surface comparison for dZ calculation can be performed using external tools like CloudCompare, which support analysis of dXY and dZ differences.
#
## =================================================
#
# DISCLAIMER:
# This script was developed with the assistance of AI tools for debugging, reviewing, and testing.
# ## =================================================


'\nThis notebook updates elevation (Z) values in a LandXML surface model using new survey data.\n\nIt performs the following steps:\n- Loads original surface points and updated survey data.\n- Matches points using nearest-neighbor search (KDTree).\n- Computes vertical differences (dZ) and filters outliers.\n- Applies cleaned dZ values to compute new Z elevations.\n- Updates the <Pnts> section in the LandXML file with new Z values.\n- Preserves the original surface structure and updates <Surface> name and description.\n- Outputs a new LandXML file for downstream use.\n\nSurface comparison and validation can be performed using external tools like CloudCompare,\nwhich support analysis of horizontal (dXY) and vertical (dZ) differences.\n'

In [1]:
#pip install scipy

In [2]:
import pandas as pd
from scipy.spatial import cKDTree
import os
import numpy as np
import xml.etree.ElementTree as ET

In [3]:
# Metadata
surface_name = "1100980_XDTM"
surface_desc = "Existing surface updated using 2025 ULS"


# directory
dir = r'P:\2025\NOGAL CANYON\02_PRODUCTION\04_QA_QC\UPDATE_SURFACE_2019'

# file pathes
csv_19 = os.path.join(dir, 'LandXML TIN PTS ORIGINAL PNEZ - 2019.csv')
csv_diff = os.path.join(dir, 'Difference_2019_vs_2025.csv')
xml_19 = os.path.join(dir, 'CN1100980_DTM_COMBINED - 2019.xml')

#Export files
new_xml = os.path.join(dir, surface_name + '.xml')
csv_temp = os.path.join(dir, "TEMP_OUTPUT.csv")

In [4]:
# Read and disaply csv files
df_19 = pd.read_csv(csv_19)
df_diff = pd.read_csv(csv_diff)

display(df_19.head())
display(df_diff.head())

,id,N,E,h
0,2,918268.752,1351793.210,5025.732
1,3,918514.208,1351659.157,5032.689
2,4,918562.168,1351681.045,5032.574
3,5,918611.191,1351700.789,5031.747
4,6,918658.552,1351721.030,5029.670


,X,Y,Z,dXY,d3D,dX,dY,dZ
0,1351793.210,918268.752,5025.731934,NaN,10.000000,NaN,NaN,NaN
1,1351659.157,918514.208,5032.688965,NaN,10.000000,NaN,NaN,NaN
2,1351681.045,918562.168,5032.574219,0.015399,0.097412,0.007202,0.013611,-0.096191
3,1351700.789,918611.191,5031.747070,0.011459,0.079106,-0.011292,0.001953,-0.078125
4,1351721.030,918658.552,5029.669922,0.020234,0.074364,0.013428,0.015137,-0.071777


In [5]:
# Build KDTree for df_diff coordinates
tree = cKDTree(df_diff[['X', 'Y', 'Z']].values)

# Find closest matches
distances, indices = tree.query(df_19[['E', 'N', 'h']].values)

# Merge based on closest match
df_19['match_index'] = indices
df_merged = df_19.merge(df_diff, left_on='match_index', right_index=True, how='left')
df_merged.drop(columns='match_index', inplace=True)

In [6]:
# Display merged df
display(df_merged.head())
df_merged[['d3D','dZ']].describe()

,id,N,E,h,X,Y,Z,dXY,d3D,dX,dY,dZ
0,2,918268.752,1351793.210,5025.732,1351793.210,918268.752,5025.731934,NaN,10.000000,NaN,NaN,NaN
1,3,918514.208,1351659.157,5032.689,1351659.157,918514.208,5032.688965,NaN,10.000000,NaN,NaN,NaN
2,4,918562.168,1351681.045,5032.574,1351681.045,918562.168,5032.574219,0.015399,0.097412,0.007202,0.013611,-0.096191
3,5,918611.191,1351700.789,5031.747,1351700.789,918611.191,5031.747070,0.011459,0.079106,-0.011292,0.001953,-0.078125
4,6,918658.552,1351721.030,5029.670,1351721.030,918658.552,5029.669922,0.020234,0.074364,0.013428,0.015137,-0.071777


,d3D,dZ
count,338090.000000,285075.000000
mean,1.935916,0.067178
std,3.540290,0.581554
min,0.000002,-9.286621
25%,0.096405,-0.157715
50%,0.312256,-0.010254
75%,0.937101,0.212891
max,10.000000,9.979980


In [7]:
# Copy to a df to perfrom cleaning
df_clean = df_merged.copy()

# Max threshold on Z value to be used for updating
DIFF_THRESHOLD = 6 
# Replace values in dZ column that are > 6 or < -6 with NaN
df_clean['dZ'] = df_clean['dZ'].where(df_clean['dZ'].between(-DIFF_THRESHOLD, DIFF_THRESHOLD), np.nan)
# Replace all NaN with 0
df_clean['dZ'] = df_clean['dZ'].fillna(0) 
df_clean[['d3D','dZ']].describe()

,d3D,dZ
count,338090.000000,338090.000000
mean,1.935916,0.053250
std,3.540290,0.505128
min,0.000002,-5.841797
25%,0.096405,-0.116699
50%,0.312256,0.000000
75%,0.937101,0.144531
max,10.000000,5.630859


In [8]:
# Update the new Z value by accouting for dZ
df_clean['newZ'] = df_clean['h'] - df_clean['dZ']
df_clean[['Z','newZ']].describe()

,Z,newZ
count,338090.000000,338090.000000
mean,4998.588915,4998.535665
std,86.048399,86.025059
min,4810.752930,4810.752732
25%,4935.523682,4935.346022
50%,5010.014648,5010.009465
75%,5059.886597,5059.818370
max,5251.084473,5251.084650


In [9]:
# Save df_clean to a CSV file
df_clean.to_csv(csv_temp, index=False)

In [10]:
## UPDATE THE LANDXML file 
# Create a new DataFrame indexed by 'id' with updated Z values
df_new = df_clean[['id', 'newZ']].set_index('id')

# Parse XML and strip namespaces
parser = ET.XMLParser(target=ET.TreeBuilder(insert_comments=True))
tree = ET.parse(xml_19, parser=parser)
root = tree.getroot()

# Remove namespace prefixes from tags
for elem in root.iter():
    if '}' in elem.tag:
        elem.tag = elem.tag.split('}', 1)[1]

# Update Z values in <Pnts>
for pnts in root.findall('.//Pnts'):
    for p in pnts.findall('P'):
        pid = int(p.attrib['id'])
        coords = p.text.strip().split()
        if pid in df_new.index:
            coords[2] = f"{df_new.loc[pid, 'newZ']:.3f}"  # Format Z to 3 decimals
            p.text = ' '.join(coords)



In [11]:
# Update Surface name and description
for surface in root.findall('.//Surface'):
    surface.set('name', surface_name) 
    surface.set('desc', surface_desc) 


In [12]:
# Save updated XML to a new file
tree.write(new_xml, encoding='utf-8', xml_declaration=True)

print(f"Updated LandXML file saved to: {new_xml}")

Updated LandXML file saved to: P:\2025\NOGAL CANYON\02_PRODUCTION\04_QA_QC\UPDATE_SURFACE_2019\1100980_XDTM.xml
